In [1]:
import numpy as np
import cv2
import os
import random
from skimage.io import imread
from skimage.transform import resize
from skimage import feature
from skimage import exposure
import matplotlib.pyplot as plt
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
import category_encoders as ce

### Variables y funciones programadas

In [2]:
#Parametros de raiz
raiz=os.getcwd()
print(raiz)

C:\Users\Ivan\Desktop\VisionArtificial


In [3]:
#Tamaño de ventana para homogeneizar las imagenes
tamanoVentana = 100

In [4]:
def tratarImagenes(directorio, X_Train, Y_Train, X_Test, Y_Test):
    os.chdir(raiz+directorio)
    aux = os.listdir()
    cont = len(aux) * 0.8
    k = 0
    for j in aux:
        img = cv2.imread(j)

        # Convertimos a escala de grises
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        #Aplicamos el filtro de la mediana que es el mejor para eliminar el ruido sal y pimienta
        img = cv2.medianBlur(img,5)
        
        #Aplicamos el filtro gaussiano para eliminar posibles ruidos restantes
        img = cv2.GaussianBlur(img,(5,5),0)

        #Redimensionamos las imagenes para tener en todas el mismo tamaño
        imagen_ampliada = cv2.resize(img, (tamanoVentana,tamanoVentana))
        
        # Detectamos los bordes con Canny
        canny = cv2.Canny(imagen_ampliada, 50, 150)
        
 
        # Buscamos los contornos
        (contornos,_) = cv2.findContours(canny.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
 
        cv2.drawContours(imagen_ampliada,contornos,-1,(0,0,255), 2)
        
        if(k<cont):
            X_Train.append(imagen_ampliada)
            Y_Train.append(j)  
        else:
            X_Test.append(imagen_ampliada)
            Y_Test.append(j)  
        k=k+1

In [5]:
#Parametros para el HOG
pixels_per_cell=(8,8)
cells_per_block=(2,2)
orientations = 9

In [6]:
def HOG(images, pixels_per_cell, cells_per_block):
    lista_hog=[]
    lista_img_hog=[]
    for i,img in enumerate(images):
        carac,img_hog=hog(img, orientations=9, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block,visualize=True, multichannel=False)
        lista_hog.append(carac)
        lista_img_hog.append(img_hog)
    return np.copy(lista_hog),lista_img_hog

# Fase 1: Entrenamiento de un clasificador de monedas

## Fase 1.1: Análisis del conjunto de datos

In [7]:
#Cargamos la imagen y la procesamos para eliminar el ruido
X_Train = []
Y_Train = []
X_Test = []
Y_Test = []

tratarImagenes('\\train\\1c',X_Train, Y_Train, X_Test, Y_Test)
tratarImagenes('\\train\\1e',X_Train, Y_Train, X_Test, Y_Test)
tratarImagenes('\\train\\2c',X_Train, Y_Train, X_Test, Y_Test)
tratarImagenes('\\train\\2e',X_Train, Y_Train, X_Test, Y_Test)
tratarImagenes('\\train\\5c',X_Train, Y_Train, X_Test, Y_Test)
tratarImagenes('\\train\\10c',X_Train, Y_Train, X_Test, Y_Test)
tratarImagenes('\\train\\20c',X_Train, Y_Train, X_Test, Y_Test)
tratarImagenes('\\train\\50c',X_Train, Y_Train, X_Test, Y_Test)


In [8]:
print("Tamaño de train") 
len(X_Train)

Tamaño de train


1041

In [9]:
print("Tamaño de test") 
len(X_Test)

Tamaño de test


256

In [12]:
#comprobando si se ha aplicado bien el filtro y la redimension

cv2.imshow("imagen",X_Train[1])
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imshow("imagen2",X_Test[255])
cv2.waitKey(0)
cv2.destroyAllWindows()

## Fase 1.2: HOG

In [ ]:
#HOG
""""
X_Train_Coin, img_Train_coin_hog = feature.hog(X_Train,orientations=orientations,pixels_per_cell=pixels_per_cell,cells_per_block=cells_per_block,transform_sqrt=True,block_norm="L1",visualize=True)
Y_Train_Coin = np.copy(Y_Train)

X_Test_Coin, img_Test_Coin = feature.hog(X_Test,orientations=orientations,pixels_per_cell=pixels_per_cell,cells_per_block=cells_per_block,transform_sqrt=True,block_norm="L1",visualize=True)
""""


In [ ]:
X_Train_Coin, img_Train_Coin_hog = HOG(X_Train, pixels_per_cell,cells_per_block)
Y_Train_Coin = np.copy(Y_Train)

X_Test_Coin, img_Test_Coin_hog = HOG(X_Test,pixels_per_cell,cells_per_block)

In [ ]:
cv2.imshow("imagen",X_Train_Coin[426])
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imshow("imagen2",X_Test_Coin[200])
cv2.waitKey(0)
cv2.destroyAllWindows()

# Fase 1.3 Entrenamiento


### CLASIFICADOR REGRESIÓN LOGÍSTICA

#### Clasificación monedas Train

In [ ]:
logistic_regression = LogisticRegression(max_iter=1000)
logistic_regression.fit(X_Train_Coin,Y_Train_Coin)

accRegLog = logistic_regression.score(X_Train_Coin,Y_Train_Coin)
print(accRegLog)

#### Clasificación monedas Test

In [ ]:
Y_Test_Coin = np.asarray(Y_Test)
accRegLogTest = logistic_regression.score(X_Test_Coin,Y_Test_Coin)
print(accRegLogTest)

### CLASIFICADOR SVM

#### Clasificacion monedas Train

In [ ]:
SVM_Coin = SVC(probability = True)
SVM_Coin.fit(X_Train_Coin,Y_Train_Coin)

In [ ]:
acc = SVM_Coin.score(X_Train_Coin,Y_Train_Coin)
print(acc)

#### Clasificacion monedas Test

In [ ]:
Y_Test_Coin = np.asarray(Y_Test)
acc = SVM_Coin.score(X_Test_Coin,Y_Test_Coin)
print(acc)

### CLASIFICADOR RED NEURONAL 

#### Clasificacion monedas Train

In [ ]:
RN_Coin = MLPClassifier(alpha=1e-5, hidden_layer_sizes=(32,64,128), random_state=1)
RN_Coin.fit(X_Train_Coin,Y_Train_Coin)
            
accRN = RN_Coin.score(X_Train_Coin,Y_Train_Coin)
print(accRN)

#### Clasificacion monedas Test

In [ ]:
Y_Test_Coin = np.asarray(Y_Test)
acc = RN_Coin.score(X_Test_Coin,Y_Test_Coin)
print(acc)

### CLASIFICADOR K-MEANS

#### Clasificacion monedas Train

In [ ]:
Kmean_Coin = KMeans(n_clusters=43,n_init = 50,max_iter = 4000)
Kmean_Coin.fit(X_Train_Coin)

accKmeans = Kmean_Coin.score(X_Train_Coin,Y_Train_Coin)
print(accKmeans)

#### Clasificacion monedas Test

In [ ]:
Y_Test_Coin = np.asarray(Y_Test)
acc = Kmean_Coin.score(X_Test_Coin,Y_Test_Coin)
print(acc)

## CLASIFICADOR RANDOM FOREST

#### Clasificacion monedas Train

In [ ]:
tree_Coin = RandomForestClassifier(max_depth=60, random_state=0, n_estimators = 100)
tree_Coin.fit(X_Train_Coin,Y_Train_Coin)

accRF = tree_Coin.score(X_Train_Coin,Y_Train_Coin)
print(accRF)

#### Clasificacion monedas Test

In [ ]:
Y_Test_Coin = np.asarray(Y_Test)
accTree = tree_Coin.score(X_Test_Coin,Y_Test_Coin)
print(accTree)
print(tree_Coin.predict(X_Test_Coin[85].reshape(1,-1)))